[Select a kernel](command:_notebook.selectKernel) to run cells.

Cas d'utilisation potentiels
Apprentissage automatique : former des modèles de classification (par exemple, Random Forest, SVM ou réseaux neuronaux) pour prédire la malignité des tumeurs.
Ingénierie des fonctionnalités : explorer les corrélations entre les fonctionnalités (par exemple, le rayon et la surface) pour identifier les principaux prédicteurs de malignité.
Visualisation des données : créez des visualisations (par exemple, des nuages de points, des cartes thermiques) pour comprendre les distributions et les relations des fonctionnalités.
Recherche médicale : Soutenir les études de pathologie computationnelle en analysant les caractéristiques nucléaires pour obtenir des informations diagnostiques.
Outil pédagogique : parfait pour enseigner les concepts de la science des données, tels que le prétraitement, l'évaluation des modèles et la validation croisée.

Catégories de fonctionnalités :
Les 30 caractéristiques sont regroupées en trois catégories principales en fonction des caractéristiques des noyaux cellulaires :

Moyenne : Valeurs moyennes des mesures (par exemple, rayon moyen, texture moyenne).
Erreur standard (ET) : Variabilité des mesures (par exemple, erreur standard du rayon, erreur standard de la surface).
Pire : Valeurs les plus grandes (pires) des mesures (par exemple, pire rayon, pire lissé).

Chaque catégorie comprend 10 mesures spécifiques :

Rayon (moyenne des distances du centre aux points du périmètre)
Texture (écart type des valeurs de niveaux de gris)
Périmètre
Zone
Lissage (variation locale des longueurs de rayon)
Compacité (périmètre² / surface - 1,0)
Concavité (gravité des parties concaves du contour)
Points concaves (nombre de portions concaves du contour)
Symétrie
Dimension fractale (« approximation du littoral » - 1)
Exemple de point de données :
voici un exemple simplifié d’une seule ligne dans l’ensemble de données :


Diagnostic ID
Rayon_moyenne
Texture_moyenne
Périmètre_moyenne
Surface_moyenne
Lissage_moyenne
…

842302
M
17,99
10,38
122,80
1001,0
0,11840
…

Interprétation : Cet échantillon (ID 842302) est malin (M), avec un rayon moyen de 17,99 unités, une texture moyenne de 10,38, etc. Les 27 colonnes restantes fournissent des mesures supplémentaires (par exemple, l'erreur type et les pires valeurs).

Structure de l'ensemble de données :
L'ensemble de données comprend 569 instances (lignes) et 32 colonnes, dont une colonne d'identification, une étiquette de diagnostic et 30 caractéristiques numériques décrivant les noyaux cellulaires. Chaque instance représente un échantillon de masse mammaire unique, dont les caractéristiques sont calculées à partir d'images FNA numérisées.
Principales colonnes :

ID : un identifiant unique pour chaque échantillon (par exemple, 842302).
Diagnostic : la variable cible, étiquetée comme suit :
M (maligne) : indique une tumeur cancéreuse.
B (bénigne) : indique une tumeur non cancéreuse.

Caractéristiques (30 colonnes) : Mesures numériques des noyaux cellulaires, telles que le rayon, la texture, le périmètre et la surface, dérivées de l'analyse d'image.

Catégories de fonctionnalités :
Les 30 caractéristiques sont regroupées en trois catégories principales en fonction des caractéristiques des noyaux cellulaires :

Moyenne : Valeurs moyennes des mesures (par exemple, rayon moyen, texture moyenne).
Erreur standard (ET) : Variabilité des mesures (par exemple, erreur standard du rayon, erreur standard de la surface).
Pire : Valeurs les plus grandes (pires) des mesures (par exemple, pire rayon, pire lissé).

Chaque catégorie comprend 10 mesures spécifiques :

Rayon (moyenne des distances du centre aux points du périmètre)
Texture (écart type des valeurs de niveaux de gris)
Périmètre
Zone
Lissage (variation locale des longueurs de rayon)
Compacité (périmètre² / surface - 1,0)
Concavité (gravité des parties concaves du contour)
Points concaves (nombre de portions concaves du contour)
Symétrie
Dimension fractale (« approximation du littoral » - 1)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.figure(figsize=(12, 8))



In [ ]:
data = pd.read_csv("Breast_cancer_dataset.csv")
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

transf = LabelEncoder()
data['diagnosis'] = transf.fit_transform(data['diagnosis'])
data['diagnosis'].value_counts()

In [ ]:
data_N = pd.isna(data)
data_N

In [ ]:
sns.heatmap(data_N,cmap='viridis')

In [ ]:
data.drop(columns=['Unnamed: 32'], inplace=True)

In [ ]:
sns.heatmap(pd.isna(data))

In [ ]:
data_calc = np.array(data)
data_calc

In [ ]:
data_cor = pd.DataFrame(data_calc).corr()
plt.figure(figsize=(12, 8))
sns.heatmap(data_cor, cmap='viridis')

In [ ]:
sns.pairplot(data, hue='diagnosis', palette='viridis')

Modelisation 

In [ ]:
from sklearn.model_selection import train_test_split


X = data.drop({'diagnosis'}, axis=1)
Y = data['diagnosis']

Y

Structure de l'ensemble de données :
L'ensemble de données comprend 569 instances (lignes) et 32 colonnes, dont une colonne d'identification, une étiquette de diagnostic et 30 caractéristiques numériques décrivant les noyaux cellulaires. Chaque instance représente un échantillon de masse mammaire unique, dont les caractéristiques sont calculées à partir d'images FNA numérisées.
Principales colonnes :

ID : un identifiant unique pour chaque échantillon (par exemple, 842302).
Diagnostic : la variable cible, étiquetée comme suit :
M (maligne) : indique une tumeur cancéreuse.
B (bénigne) : indique une tumeur non cancéreuse.

Caractéristiques (30 colonnes) : Mesures numériques des noyaux cellulaires, telles que le rayon, la texture, le périmètre et la surface, dérivées de l'analyse d'image.

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:

from catboost import CatBoostClassifier,Pool,cv,MetricVisualizer
model = CatBoostClassifier(iterations=2000, learning_rate=0.03, depth=6, eval_metric='AUC',l2_leaf_reg= 5,border_count=128,bootstrap_type='Bayesian', loss_function='Logloss', verbose=10,bagging_temperature=1, auto_class_weights='Balanced',random_seed=42)
model.fit(x_train, y_train, eval_set=(x_test, y_test),plot = True,early_stopping_rounds=100,use_best_model=True)
model.get_best_score()
model.get_best_iteration()

In [ ]:
model.predict(y_test)

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
y_preds = model.predict(x_test)


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_preds))
conf_matrix =confusion_matrix(y_test, y_preds)
sns.heatmap(conf_matrix, fmt='d', cmap='Blues', annot=True, xticklabels=transf.classes_, yticklabels=transf.classes_)
print(classification_report(y_test, y_preds, target_names=transf.classes_))
print("ok")

In [ ]:
x_train.columns

test sur les donnes inconu 

In [ ]:
x_data = pd.read_csv('test_dataset_full.csv')
x_data = x_data.drop({'diagnosis'}, axis=1)
x_data.head()

In [ ]:
df_real = pd.read_csv("test_dataset_full.csv")
X_real = df_real.drop(['diagnosis'], axis=1)
y_real = df_real['diagnosis']

# Prédictions
y_pred = model.predict(X_real)
y_proba = model.predict_proba(X_real)[:, 1]

print("classification: ",y_pred)
print("probability: ",y_proba.max()*100)

print(classification_report(y_real, y_pred))
conf_matrix_real = confusion_matrix(y_real, y_pred)
sns.heatmap(conf_matrix_real, fmt='d', cmap='Blues', annot=True)
